In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/815.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/491.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/718.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/709.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/379.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/780.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/248.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/480.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/236.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/630.jpg_0.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/771.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/675.png
/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test/DROWSY/340.png
/kaggle/input/drowsy-detection-dataset/Drowsy

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# **تعريف الثوابت**
SEQUENCE_LENGTH = 15  # عدد الإطارات لكل تسلسل
IMG_SIZE = (224, 224)  # حجم الصورة لكل إطار
BATCH_SIZE = 8  # عدد التسلسلات في كل batch (يجب أن يكون مضاعفًا لـ SEQUENCE_LENGTH)
IMAGES_PER_BATCH = SEQUENCE_LENGTH * BATCH_SIZE  # إجمالي عدد الصور في كل batch

# **تحميل البيانات**
data_dir = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=IMAGES_PER_BATCH,  # تحميل عدد مناسب من الصور
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=IMAGES_PER_BATCH,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# **تحويل الصور إلى تسلسلات (Sequences)**
def create_sequence(generator):
    while True:
        batch = next(generator)
        images, labels = batch

        num_samples = images.shape[0]  # عدد الصور المحملة

        # **تأكد من أن عدد الصور يمكن تقسيمه إلى تسلسلات**
        if num_samples % SEQUENCE_LENGTH != 0:
            continue  # تجاوز هذه الدورة من اللوب إذا لم يكن العدد متوافقًا

        # تقسيم الصور إلى مجموعات من SEQUENCE_LENGTH
        X_seq = np.reshape(images, (-1, SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3))
        y_seq = np.reshape(labels, (-1, SEQUENCE_LENGTH))[:, 0]  # أخذ التصنيف الأول فقط
        
        yield X_seq, y_seq

train_generator = create_sequence(train_gen)
val_generator = create_sequence(val_gen)

Found 4688 images belonging to 2 classes.
Found 1171 images belonging to 2 classes.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

print(f"عدد عينات التدريب: {train_generator.samples}")
print(f"عدد عينات التحقق: {val_generator.samples}")


Found 5859 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.
عدد عينات التدريب: 5859
عدد عينات التحقق: 1483


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# **إعداد البيانات**
IMG_SIZE = (96, 96)
SEQUENCE_LENGTH = 8
BATCH_SIZE = 4

# تحميل البيانات باستخدام ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train",
    target_size=IMG_SIZE,
    batch_size=1,  # سيتم تجميع الإطارات لاحقًا
    class_mode="binary",
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/test",
    target_size=IMG_SIZE,
    batch_size=1,  # سيتم تجميع الإطارات لاحقًا
    class_mode="binary",
    shuffle=False
)

# ** إنشاء مولّد مخصص لإنشاء التسلسلات**
def custom_data_generator(image_generator, sequence_length):
    """مولد بيانات يقوم بتجميع الإطارات في تسلسلات"""
    frames, labels = [], []
    
    while True:
        for img, label in image_generator:
            frames.append(img[0])  # استخراج الصورة من الـ batch
            labels.append(label[0])
            
            if len(frames) == sequence_length:
                yield np.array(frames).reshape(1, sequence_length, IMG_SIZE[0], IMG_SIZE[1], 3), np.array(labels[-1]).reshape(1, 1)
                frames, labels = [], []  # إعادة تعيين القوائم

train_gen = custom_data_generator(train_generator, SEQUENCE_LENGTH)
val_gen = custom_data_generator(val_generator, SEQUENCE_LENGTH)

# ** بناء النموذج**
base_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')
base_model.trainable = False  # تجميد أوزان MobileNetV2

model = Sequential([
    TimeDistributed(base_model, input_shape=(SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)),
    TimeDistributed(GlobalAveragePooling2D()),  # تقليل حجم المعاملات
    LSTM(32),  # ت ال LSTM
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# ** إعدادات التدريب**
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ** طباعة ملخص النموذج**
model.summary()

# ** ضبط الـ Callbacks**
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)

# ** تدريب النموذج**
model.fit(train_gen,
          validation_data=val_gen,
          epochs=15,
          steps_per_epoch=len(train_generator) // SEQUENCE_LENGTH,
          validation_steps=len(val_generator) // SEQUENCE_LENGTH,
          callbacks=[early_stopping, lr_scheduler])


Found 5859 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 8, 3, 3, 1280)       │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 8, 1280)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 32)                  │         168,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,427,137 (9.26 MB)

 Trainable params: 169,153 (660.75 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/15
732/732 ━━━━━━━━━━━━━━━━━━━━ 75s 59ms/step - accuracy: 0.6526 - loss: 0.6073 - val_accuracy: 0.8973 - val_loss: 0.2511 - learning_rate: 5.0000e-04
Epoch 2/15
732/732 ━━━━━━━━━━━━━━━━━━━━ 31s 43ms/step - accuracy: 0.8494 - loss: 0.3478 - val_accuracy: 0.9514 - val_loss: 0.1766 - learning_rate: 5.0000e-04
Epoch 3/15
686/732 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.8918 - loss: 0.2587

In [ ]:
model.save("model.h5")


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# تحميل النموذج المدرب
model = tf.keras.models.load_model("model.h5")

# تصنيفات الفئات
class_names = ["نائم", "مستيقظ"]

# دالة لتوقع الفئة من صورة واحدة وعرضها
def predict_and_show_image(image_path):
    # تحميل الصورة وتغيير الحجم
    img = image.load_img(image_path, target_size=(96, 96))
    img_array = image.img_to_array(img) / 255.0  # تحويل لمصفوفة وتقسيم على 255

    # تكرار الصورة 8 مرات لجعل الشكل متوافقًا مع النموذج
    img_sequence = np.array([img_array] * 8)  # الآن شكله (8, 96, 96, 3)
    img_sequence = np.expand_dims(img_sequence, axis=0)  # يصبح (1, 8, 96, 96, 3)

    # إجراء التوقع
    prediction = model.predict(img_sequence)[0][0]
    label = class_names[int(prediction > 0.5)]
    confidence = prediction if prediction > 0.5 else 1 - prediction

    # قراءة الصورة لعرضها باستخدام OpenCV
    img_cv = cv2.imread(image_path)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)  # تحويل الألوان من BGR إلى RGB

    # عرض الصورة باستخدام matplotlib
    plt.figure(figsize=(5, 5))
    plt.imshow(img_cv)
    plt.axis("off")  # إخفاء المحاور
    plt.show()

    # **طباعة النتيجة في الكونسول بدلًا من وضعها على الصورة**
    print(f"🔹 التوقع: {label} (ثقة: {confidence:.2%})")

# **استخدام الكود لاختبار صورة**
image_path = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train/NATURAL/860.png"
predict_and_show_image(image_path)


In [ ]:
image_path = "/kaggle/input/drowsy-detection-dataset/Drowsy_datset/train/DROWSY/1502.png"
predict_and_show_image(image_path)